<a href="https://colab.research.google.com/github/Brice12347/winterproject/blob/main/WinterProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#i started off by following a tutorial of how to import data from kaggle
#https://www.youtube.com/watch?v=57N1g8k2Hwc
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"bricepatchou","key":"ab7aefc9f4acdb988f5d7c76529feb94"}'}

In [ ]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d samuelcortinhas/muffin-vs-chihuahua-image-classification

100% 473M/474M [00:16<00:00, 33.2MB/s]
100% 474M/474M [00:16<00:00, 29.3MB/s]


In [ ]:
#I now have the data i just have to get all 5000ish images out of the file
!unzip muffin-vs-chihuahua-image-classification.zip

Archive:  muffin-vs-chihuahua-image-classification.zip
replace test/chihuahua/img_0_1071.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test/chihuahua/img_0_1071.jpg  
replace test/chihuahua/img_0_1074.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!mkdir data

In [ ]:
!mkdir newstuff

In [ ]:
#I wanted to add files from my google drive to the directory but i found out pretty fast that I did not know how to do it
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#importing libraries and functions for pytorch
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True
plt.ion() 

In [ ]:
#downloading the pretrained resnet model
model_resnet18 = torch.hub.load('pytorch/vision', 'resnet18', pretrained=True)


Using cache found in /root/.cache/torch/hub/pytorch_vision_main


ImportError: ignored

In [ ]:
#i would prefer to use a GPU but if I cant, use a cpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
#transforming the images for the model
#these are numbers that i stole from like 3 youtube videos and the notebook. one of the videos said that the numbers are from the resnet website and show the optimal numbers for thefunction
#https://www.youtube.com/watch?v=FaW9JCSJn2s&t=500s at 1:32
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'data/newstuff'
#put the data into dataloaders
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
#create a training model
#i got the training model from the Notebook and https://www.youtube.com/watch?v=K0lWSB2QoIQ
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()
    #the model weights and best accuracy
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    #for every epoch
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        #for each phase/ whether that be train or val
        # if the phase is train then the mode will train and if it is evaluate it will evaluate
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  
            else:
                model.eval()  

            running_loss = 0.0
            running_corrects = 0

            #for input and label in the dataloader 
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                #make the gradient 0
                optimizer.zero_grad()

                # if the phase is train record accuracies and inaccuracies
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # if the phase is train, use the inaccuracies to backpropagate
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # look at the accuracy of each epoch. if the current accuracy is greater than the best accuracy it is now the best accuracy and we deep copy the model weights of the epoch of the best accuracy
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
# load the resnet model
model_ft = models.resnet18(pretrained=True)
# modify the last layer to output 2 as there are two classes
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)

# save model to device
model_ft = model_ft.to(device)

# define loss for binary classification
criterion = nn.CrossEntropyLoss()

# create optimizer
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# decelerate the learning rate gradually. every epoch(im only doing 5) the learning rate will decrease by 5 percent. this is done to not overfit
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=1, gamma=0.05)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
# train the model!
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=5)

Epoch 0/4
----------
train Loss: 0.3565 Acc: 0.8656
val Loss: 0.0215 Acc: 0.9924

Epoch 1/4
----------
train Loss: 0.2260 Acc: 0.9172
val Loss: 0.0253 Acc: 0.9924

Epoch 2/4
----------
train Loss: 0.2237 Acc: 0.9165
val Loss: 0.0223 Acc: 0.9932

Epoch 3/4
----------
train Loss: 0.2178 Acc: 0.9199
val Loss: 0.0208 Acc: 0.9941

Epoch 4/4
----------
train Loss: 0.2297 Acc: 0.9108
val Loss: 0.0308 Acc: 0.9916

Training complete in 7m 25s
Best val Acc: 0.994088
